## <p style="text-align: center;">Numerical Algorithms - Homework Assignment 3</p>
**<p style="text-align: center;">VU Numerical Algorithms, summer semester 2018. Due to 17.06.2018. </p>**

### Programming Exercises

#### Effects of Preconditioners on Conjugate Gradient (12 points)

Implement the conjugate gradient method *CG* and experimentally investigate the effect of different preconditioners on its convergence. Please compare standard *CG* and preconditioned *CG* (*PCG*) for three given symmetric positive definite test problems in terms of the number of iterations and in terms of the runtime (including the time for
computing and applying the preconditioner!) until convergence. Show the convergence histories (norm of relative residual vs. iteration number) graphically.

* Implement standard CG efficiently (_do not use the CG implementation available in Octave!_). In particular, store the sparse matrix in a sparse matrix format
* Use the following preconditioners:
Diagonal preconditioner
  * Block diagonal preconditioner
  * Incomplete Cholesky factorization with no fill-in
  * Incomplete Cholesky factorization with threshold dropping: experiment with different thresholds and discuss the effect of the choice of threshold.
* Test matrices: Please use (at least) the following three test matrices from the
"Matrix Market" (http://math.nist.gov/MatrixMarket/) for your experiments:
  * http://math.nist.gov/MatrixMarket/data/Harwell-Boeing/lanpro/nos5.html
  * http://math.nist.gov/MatrixMarket/data/Harwell-Boeing/lanpro/nos6.html
  * http://math.nist.gov/MatrixMarket/data/misc/cylshell/s3rmt3m3.html

In [ ]:
%plot -f svg

% Import routines needed to execute assignment.
source("source/assignment3.m")
% Import routines for reading matrix market (.mtx) format.
% Source: https://math.nist.gov/MatrixMarket/mmio/matlab/mmiomatlab.html.
source("source/mminfo.m")
source("source/mmread.m")

% Execute code for programming assignment.
function execute()
    res = struct();
    
    % Iterate over test matrices.
    filenames = {"nos5.mtx"; "nos6.mtx"; "s3rmt3m3.mtx"};
    fileAlphas = [0.5 0 774]; % s3: 1032
    fileBlocksizes = [2 4 64];
    fileThresholds = [0.2 0.2 3];
    
    for i = 1:length(filenames)
        % Load file.
        [A, rows, cols, entries, rep, field, symm] = mmread(["data/" filenames{i}]);

        % Prepare data structure for evaluation metrics.
        res.(filenames{i}).st = struct();
        
        % ----------------------------------------------------
        % 1. Apply standard CG.
        % ----------------------------------------------------
        
        startTime = time();
        [
            res.(filenames{i}).st.numberOfIterations, ...
            res.(filenames{i}).st.runTime, ...
            res.(filenames{i}).st.relativeResidualHistory
        ] = applyCG(A, 10^-8);
        res.(filenames{i}).st.runTime = time() - startTime();
        
        % ----------------------------------------------------
        % 2. Apply CG with preconditioner.
        % ----------------------------------------------------
        
        pcNames = {"diag"; "block"; "cholesky_nofi"; "cholesky_drop"};
        for j = 1:length(pcNames)
            res.(filenames{i}).(pcNames{j}) = struct();
            
            % Configure preconditioner.
            % Note: Parameters used here were determined in prior experiments (see documentation).
            precondConfig = struct();
            precondConfig.method = pcNames{j};
            % Set predetermined/fixed and matrix-specific alpha.
            precondConfig.alpha = fileAlphas(i);
            precondConfig.stepsize = fileBlocksizes(i);
            precondConfig.dropThreshold = fileThresholds(i);
            
            % Generate matrix for preconditioning.
            M = generatePreconditioning(A, precondConfig);
            
            % Apply CG with preconditioning.
            startTime = time();
            [
                res.(filenames{i}).(pcNames{j}).numberOfIterations, ...
                res.(filenames{i}).(pcNames{j}).runTime, ...
                res.(filenames{i}).(pcNames{j}).relativeResidualHistory
            ] = applyCG(A, 10^-8, inv(M));
            res.(filenames{i}).(pcNames{j}).runTime = time() - startTime();
            
            pcNames{j}
        end
    end
end

% Execute task.
execute()

ans = diag
-------ans = block
-------ans = cholesky_nofi
-------ans = cholesky_drop
-------ans = diag
-------ans = block
-------ans = cholesky_nofi
-------ans = cholesky_drop
-------ans = diag
-------ans = block
-------ans = cholesky_nofi
-------panic: Segmentation fault -- stopping myself...


In [ ]:
%plot -f svg

% Import routines needed to execute assignment.
source("source/assignment3.m")
% Import routines for reading matrix market (.mtx) format.
% Source: https://math.nist.gov/MatrixMarket/mmio/matlab/mmiomatlab.html.
source("source/mminfo.m")
source("source/mmread.m")

% Investigate effect of threshold for PC with 
% incomplete Cholesky factorization with threshold 
% dropping.
function executeParameterGridSearch()
    starttime = time();
    printf("Starting grid search.\n")
    
    % Define matrix file names.
    filenames = {"s3rmt3m3.mtx"; "nos5.mtx"; "nos6.mtx"};
    % Determined by line search.
    fileAlphas = [1238.972321 0.16 0];
    
    % Create structure for holding results.
    res = struct();
    
    % ----------------------------------------
    % 1. Grid search for block-diagonale M.
    % ----------------------------------------
    %{
    printf("\n\n* Grid seach for block-diagonale (block size).")
    
    precondConfig = struct();
    precondConfig.method = "block";
    
    
    % Iterate over test matrices.
    for i = 1:length(filenames)
        printf("\n  - %s", filenames{i});
        fflush(stdout);
        
        res.(filenames{i}) = struct();
        res.(filenames{i}).numberOfIterations = [];
        res.(filenames{i}).runtime = [];
        res.(filenames{i}).relativeResidualHistory = [];
        
        % Load file.
        [A, rows, cols, entries, rep, field, symm] = mmread(["data/" filenames{i}]);        
        
        % Compute CG with block-diag. M with different block sizes.
        stepsizes = [2 4 8 16 32 64:round(size(A)(1) / 20):size(A)(1) / 4];
        for stepsize = stepsizes
            precondConfig.stepsize = stepsize;
            M = generatePreconditioning(A, precondConfig);
            [numIter, runtime, relResHistory] = applyCG(A, 10^-8, inv(M));
            res.(filenames{i}).numberOfIterations = [res.(filenames{i}).numberOfIterations numIter];
            res.(filenames{i}).runtime = [res.(filenames{i}).runtime runtime];
        end
        
        [opt_runtime opt_idx] = min(res.(filenames{i}).runtime);
        printf("    Optimal block size found: %f", stepsizes(opt_idx));
    end
    %}
    
    % ----------------------------------------
    % 2. Grid search for incomplete Cholesky 
    % factorization with no fill-in.
    % ----------------------------------------
    printf("\n\n* Grid seach for incomplete Cholesky factorization with no fill-in (alpha).")
    
    precondConfig = struct();
    precondConfig.method = "cholesky_nofi";
    
    % {
    % Iterate over test matrices.
    for i = 1:length(filenames)
        printf("\n  - %s", filenames{i})
    
        res.(filenames{i}) = struct();
        res.(filenames{i}).numberOfIterations = [];
        res.(filenames{i}).runtime = [];
        res.(filenames{i}).relativeResidualHistory = [];
        
        % Load file.
        [A, rows, cols, entries, rep, field, symm] = mmread(["data/" filenames{i}]);
        
        % Compute CG with block-diag. M with different block sizes.
        % For calculation of upperAlphaLimit: https://www.mathworks.com/help/matlab/ref/ichol.html#btuq275-3.
        upperAlphaLimit = abs(max(sum(abs(A), 2) ./ diag(A)) - 2);
        alphas = [upperAlphaLimit upperAlphaLimit / 2];
        for alpha = alphas
            precondConfig.alpha = alpha;
            
            M = generatePreconditioning(A, precondConfig);
            [numIter, runtime, relResHistory] = applyCG(A, 10^-8, inv(M));
            res.(filenames{i}).numberOfIterations = [res.(filenames{i}).numberOfIterations numIter];
            res.(filenames{i}).runtime = [res.(filenames{i}).runtime runtime];
        end

        [opt_runtime opt_idx] = min(res.(filenames{i}).runtime);
        printf("    Optimal alpha found: %f", alphas(opt_idx));
    end    
    % }
    
    % ----------------------------------------
    % 3. Grid search for incomplete Cholesky 
    % factorization with drop threshold for 
    % fill-in.
    % ----------------------------------------
    %{
    printf("\n\n* Grid seach for incomplete Cholesky factorization with fill-in dropping threshold (alpha, dropping threshold).")
    
    precondConfig = struct();
    precondConfig.method = "cholesky_drop";
    
    % Iterate over test matrices.
    for i = 1:length(filenames)
        printf("\n  - %s", filenames{i})
        fflush(stdout);
    
        res.(filenames{i}) = struct();
        res.(filenames{i}).numberOfIterations = [];
        res.(filenames{i}).runtime = [];
        res.(filenames{i}).relativeResidualHistory = [];
        
        % Load file.
        [A, rows, cols, entries, rep, field, symm] = mmread(["data/" filenames{i}]);        
        
        % Set alpha (determined in previous step).
        precondConfig.alpha = fileAlphas(i);
        
        % Compute CG with Cholesky factorization and fixed alpha. Vary dropping threshold.
        % Note: Dropping threshold is used as: abs (L(i,j)) >= droptol * norm (A(j:end, j), 1)
        thresholds = [0.1 0.2 0.3 0.4 0.5 1 2 3 5 10 15 20];
        for threshold = thresholds
            precondConfig.dropThreshold = threshold;
            
            M = generatePreconditioning(A, precondConfig);
            [numIter, runtime, relResHistory] = applyCG(A, 10^-8, inv(M));
            res.(filenames{i}).numberOfIterations = [res.(filenames{i}).numberOfIterations numIter];
            res.(filenames{i}).runtime = [res.(filenames{i}).runtime runtime];
        end
        
        [opt_runtime opt_idx] = min(res.(filenames{i}).runtime);
        printf("    Optimal drop treshold found: %f", thresholds(opt_idx));
        
        % -----------------------------------
        % Plot results.
        % -----------------------------------
        
        % Thresholds vs. iterations.
        figure('Position',[0, 0, 450, 350])
        grid on
        hold on
        plot(thresholds, res.(filenames{i}).numberOfIterations, 'markersize', 3, '3; Thresholds vs. iterations;o-');
        title (["" filenames{i} ": Comparison of thresholds and iterations"], 'fontsize', 16);
        xlabel('Thresholds');
        ylabel('Number of iterations');   
        % Thresholds vs. runtimes.
        figure('Position',[0, 0, 450, 350])
        grid on
        hold on
        plot(thresholds, res.(filenames{i}).runtime, 'markersize', 3, '2; Thresholds vs. runtimes;x-');
        title (["" filenames{i} ": Comparison of thresholds and runtimes"], 'fontsize', 16);
        xlabel('Thresholds');
        ylabel('Runtimes');           
    end
    %}
    
    printf("\n\nGrid search done. Time: %i\n", time() - starttime);
end

% Execute parameter grid/line search to find decent values for
%    - alpha
%    - stepsize
%    - dropThreshold.
% Note that this search is variant towards the investigated PC-method.
executeParameterGridSearch()

Starting grid search.


* Grid seach for incomplete Cholesky factorization with no fill-in (alpha).
